In [4]:
import pandas as pd 
import os
import Bio
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import ClustalwCommandline
import collections
from Bio import AlignIO
import random
import numpy as np
import sys

#sys.path.append(os.path.abspath('/home/nimar/PLP_directRNA_design_V2/PLP_directRNA_design/'))
#from PLP_directRNA_design import probedesign as plp

In [ ]:
number_of_seqs= 10 # number of regions mapped/PLP
final_designed=5 # number of PLPs designed at the end/gene
plp_length=30
mismatches = 6


In [21]:
# Define IUPAC nucleotide codes for mismatches
IUPAC_CODES = {
    "R": ["A", "G"],
    "Y": ["C", "T"],
    "S": ["G", "C"],
    "W": ["A", "T"],
    "K": ["G", "T"],
    "M": ["A", "C"],
    "B": ["C", "G", "T"],
    "D": ["A", "G", "T"],
    "H": ["A", "C", "T"],
    "V": ["A", "C", "G"],
    "N": ["A", "C", "G", "T"]  # Any base
}

ligation_junctions_dict = {
    'TA': 'preferred', 'GA': 'preferred', 'AG': 'preferred',
    'TT': 'neutral', 'CT': 'neutral', 'CA': 'neutral',
    'TC': 'neutral', 'AC': 'neutral', 'CC': 'neutral',
    'TG': 'neutral', 'AA': 'neutral',
    'CG': 'non-preferred', 'GT': 'non-preferred',
    'GG': 'non-preferred', 'GC': 'non-preferred'
}

def evaluate_ligation_junction(targets, iupac_mismatches=None, plp_length=30):
    """
    Evaluates the ligation junction of a probe and introduces mismatches if needed.

    Args:
        probe_seq (str): The probe sequence.
        iupac_mismatches (list of tuples): List of positions and IUPAC codes to introduce mismatches.
                                            Example: [(5, 'R'), (10, 'Y')]
        plp_length (int): Probe length (default: 30).

    Returns:
        tuple: (updated probe sequence, ligation junction category)
    """
    # Extract the ligation junction (2 bases around the center of the probe)
    junction_position = int((plp_length / 2) - 1)
    for idx in targets.index:
        #print(f"Index: {idx}, Row Data: {targets.loc[idx]}")
        probe_seq = targets.loc[idx]['Sequence']
        ligation_junction = probe_seq[junction_position] + probe_seq[junction_position + 2]

        # Determine the ligation status
        ligation_status = ligation_junctions_dict.get(ligation_junction, "non-preferred")
        targets.loc[idx]['ligation_status'] = ligation_status

        # Apply IUPAC mismatches if provided
        num_mismatches = len(iupac_mismatches)
        if iupac_mismatches:
            for r in range(1, num_mismatches + 1):
                for subset in itertools.combinations(range(num_mismatches), r):
                    selected_mismatches = [iupac_mismatches[i] for i in subset]
                    replacement_options = [IUPAC_CODES[symbol] for pos, symbol in selected_mismatches]
                    # Generate all possible combinations of replacements
                    for replacement in itertools.product(*replacement_options):
                        new_probe_seq = list(probe_seq)
                        new_id_suffix = []
                        for (pos, iupac_symbol), new_base in zip(selected_mismatches, replacement):
                            new_probe_seq[pos] = new_base
                            new_id_suffix.append(f"{pos}_{iupac_symbol}_{new_base}")
                        new_probe_seq = "".join(new_probe_seq)
                        new_probe_id = f"{idx}|{'_'.join(new_id_suffix)}"
                        # Revaluate the ligation junction
                        new_ligation_junction = new_probe_seq[junction_position] + new_probe_seq[junction_position + 2]
                        new_ligation_status = ligation_junctions_dict.get(new_ligation_junction, "non-preferred")
                        new_row = targets.loc[idx].copy()
                        new_row['Sequence'] = new_probe_seq
                        new_row['Ligation junction'] = new_ligation_status
                        targets.loc[new_probe_id] = new_row
            
    
    return targets


In [60]:
prob_seq = 'AGAGGAGGAGAGCAAAGAGGCCAGTGCTCT'
iupac_mismatches = [(5, 'R'), (10, 'Y')]
plp_length = 30
probeSeq, ligation_status = evaluate_ligation_junction(prob_seq, iupac_mismatches, plp_length)
print(probeSeq, ligation_status)


TypeError: 'builtin_function_or_method' object is not iterable

In [61]:
import os
import pandas as pd
junction_position = int((plp_length / 2) - 1)

targets = pd.read_csv('../targets.txt', sep='\t', index_col=0)
for idx in targets.index:
    #print(f"Index: {idx}, Row Data: {targets.loc[idx]}")
    probe_seq = targets.loc[idx]['Sequence']
    ligation_junction = probe_seq[junction_position] + probe_seq[junction_position + 2]

    # Determine the ligation status
    ligation_status = ligation_junctions_dict.get(ligation_junction, "non-preferred")
    targets.loc[idx]['ligation_status'] = ligation_status
    #print(f"Ligation junction: {ligation_junction} ({ligation_status})")
    # Apply IUPAC mismatches if provided
    if iupac_mismatches:
        probe_seq = list(probe_seq)  # Convert to list for mutability
        for pos, iupac_symbol in iupac_mismatches:
            if 0 <= pos < len(probe_seq) and iupac_symbol in IUPAC_CODES:
                for base in IUPAC_CODES[iupac_symbol]:
                    new_probe_seq = probe_seq.copy()
                    new_probe_seq[pos] = base
                    new_probe_seq = "".join(new_probe_seq)  # Convert back to string
                    new_probe_id = f"{idx}|{pos}_{iupac_symbol}_{base}"
                    new_row = targets.loc[idx].copy()
                    new_row['Sequence'] = new_probe_seq
                    new_row['Ligation junction'] = ligation_junctions_dict.get(
                        new_probe_seq[junction_position] + new_probe_seq[junction_position + 2], "non-preferred"
                    )
                    targets.loc[new_probe_id] = new_row


targets

/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_81765/2685275217.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.loc[idx]['ligation_status'] = ligation_status
/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_81765/2685275217.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.loc[idx]['ligation_status'] = ligation_status
/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_81765/2685275217.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,Gene,Region,Sequence,GC,Coverage,Ligation junction
Probe_id,,,,,,
Prlh|90880853-90880882,Prlh,1:90880853-90880882,GTGCTTGCTGCTGCTAGGCTTAGTCCTCCC,60.000000,1.0,neutral
Prlh|90880854-90880883,Prlh,1:90880854-90880883,TGCTTGCTGCTGCTAGGCTTAGTCCTCCCA,56.666667,1.0,preferred
Prlh|90880855-90880884,Prlh,1:90880855-90880884,GCTTGCTGCTGCTAGGCTTAGTCCTCCCAG,60.000000,1.0,non-preferred
Prlh|90880856-90880885,Prlh,1:90880856-90880885,CTTGCTGCTGCTAGGCTTAGTCCTCCCAGG,60.000000,1.0,non-preferred
Prlh|90880857-90880886,Prlh,1:90880857-90880886,TTGCTGCTGCTAGGCTTAGTCCTCCCAGGA,56.666667,1.0,neutral
...,...,...,...,...,...,...
Grik2|49659291-49659320|10_Y_T,Grik2,10:49659291-49659320,TAAAGTCCTGTTCTGCTTGTTGTGGATCGG,50.000000,6.0,non-preferred
Grik2|49659292-49659321|5_R_A,Grik2,10:49659292-49659321,AAAGTACTGCTCTGCTTGTTGTGGATCGGA,50.000000,6.0,neutral
Grik2|49659292-49659321|5_R_G,Grik2,10:49659292-49659321,AAAGTGCTGCTCTGCTTGTTGTGGATCGGA,50.000000,6.0,neutral


In [59]:
import itertools

num_mismatches = len(iupac_mismatches)
for subset in itertools.combinations(range(num_mismatches), 2):
    selected_mismatches = [iupac_mismatches[i] for i in subset]
    replacement_options = [IUPAC_CODES[symbol] for pos, symbol in selected_mismatches]
    for replacement in itertools.product(*replacement_options):
        print(replacement)
    #print(replacement_options)
    #for pos, symbol in selected_mismatches:
        #print(pos, symbol)
        #print(IUPAC_CODES[symbol])



('A', 'C')
('A', 'T')
('G', 'C')
('G', 'T')


In [10]:
import pandas as pd 
import os
cur_path = os.chdir('/Users/nimra236/Dropbox/NBIS/Projects/PLP_directRNA_design_V2/')
print(cur_path)

selected_features = 'extract_features_output.txt'
selected_features= pd.read_csv(selected_features, sep='\t')
print(selected_features.columns)

None
Index(['seqname', 'start', 'end', 'strand', 'gene_name', 'transcript_id',
       'coverage', 'region'],
      dtype='object')


# Extracting sequence from gtf/fasta

In [26]:
def extract_transcripts_biopython(genome_fasta, annotation_gtf, output_fasta):
    """
    Extracts coding sequences (CDS) of transcripts using Biopython.
    """
    start_time = time.time()
    
    # Load genome sequences
    genome = SeqIO.to_dict(SeqIO.parse(genome_fasta, "fasta"))
    
    # Create GTF database
    db = gffutils.create_db(annotation_gtf, dbfn='annotation.db', force=True, keep_order=True, 
                            merge_strategy="merge", disable_infer_genes=True)
    
    transcripts = defaultdict(str)
    
    # Extract CDS sequences for each transcript
    for transcript in db.features_of_type("transcript"):
        chrom = transcript.seqid
        cds_seq = ""
        
        for cds in db.children(transcript, featuretype="CDS", order_by="start"):
            start = cds.start - 1  # Convert to 0-based index
            end = cds.end
            
            if chrom in genome:
                seq_fragment = genome[chrom].seq[start:end]
                if cds.strand == "-":
                    seq_fragment = seq_fragment.reverse_complement()
                cds_seq += str(seq_fragment)
        
        if cds_seq:
            transcripts[transcript.id] = cds_seq
    
    # Write output
    with open(output_fasta, "w") as f:
        for tid, seq in transcripts.items():
            f.write(f">{tid}\n{seq}\n")
    
    elapsed_time = time.time() - start_time
    print(f"Biopython CDS extraction completed in {elapsed_time:.2f} seconds. Output saved to: {output_fasta}")


In [23]:
extract_transcripts_biopython('../data/Mus.fa', annotation_gtf='../data/tmp.gtf', output_fasta='tmp.fasta')

Biopython CDS extraction completed in 4.20 seconds. Output saved to: tmp.fasta


# Find targets

In [ ]:
import sys
import os
import pandas as pd
import dnaio
from Bio import SeqIO
from Bio.Seq import Seq
from cutadapt.adapters import BackAdapter
from dataclasses import dataclass
from io import StringIO
import csv

import pandas as pd 
from pandas import DataFrame
import Bio
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import collections
from Bio import AlignIO
import random
import numpy as np
import argparse
import re
import os 
import multiprocessing
import subprocess
import itertools


import tqdm

# Dictionaries

## IUPAC dictionary
IUPAC_CODES = {
    "R": ["A", "G"],
    "Y": ["C", "T"],
    "S": ["G", "C"],
    "W": ["A", "T"],
    "K": ["G", "T"],
    "M": ["A", "C"],
    "B": ["C", "G", "T"],
    "D": ["A", "G", "T"],
    "H": ["A", "C", "T"],
    "V": ["A", "C", "G"],
    "N": ["A", "C", "G", "T"],  
    "A": ["A"],
    "C": ["C"],
    "G": ["G"],
    "T": ["T"]
}

## Ligation junction dictionary
ligation_junctions_dict = {'TA': 'preferred',
                        'TA': 'preferred',
                        'GA': 'preferred',
                        'AG': 'preferred',
                        'TT': 'neutral',
                        'CT': 'neutral',
                        'CA': 'neutral',
                        'TC': 'neutral',
                        'AC': 'neutral',
                        'CC': 'neutral',
                        'TG': 'neutral',
                        'AA': 'neutral', 
                        'CG': 'non-preferred', 
                        'GT': 'non-preferred',
                        'GG': 'non-preferred',
                        'GC': 'non-preferred'}


@dataclass
class Alignment:
    query_name: str
    target_name: str
    target_start: int
    target_end: int
    mismatches: int
    query_sequence: str
    target_sequence: str

from tqdm import tqdm  # Import tqdm for progress bar

def find_probes_in_targets(targets_df, reference_fasta, max_errors=1, output_file=None):
    """
    Function to check specificity of extracted probes against a reference genome.
    Uses Cutadapt's aligner for finding target regions.

    Args:
        targets_df (DataFrame): DataFrame containing extracted probe sequences.
        reference_fasta (str): Path to the reference genome FASTA.
        max_errors (int): Maximum number of allowed mismatches.
        output_file (str, optional): Path to save the results as a CSV file.

    Returns:
        DataFrame: A DataFrame containing matched probe alignments.
    """
    results = []

    with dnaio.open(reference_fasta) as references:
        references = list(references)  # Load references into a list
        total_probes = len(targets_df)  # Get total probe count

        with tqdm(total=total_probes, desc="Testing probes for specificity", unit=" alignments") as pbar:
            for reference_record in references:
                ref_id = reference_record.id
                ref_seq = reference_record.sequence

                for _, row in targets_df.iterrows():
                    probe_id = row["Probe_id"]
                    probe_seq = row["Sequence"]

                    adapter = BackAdapter(probe_seq, max_errors=max_errors, min_overlap=len(probe_seq), indels=False)
                    aligner = adapter.aligner

                    # Forward strand search
                    for t_start, t_end, errors, target_seq in find_all(ref_seq, aligner):
                        results.append(Alignment(
                            query_name=probe_id,
                            target_name=ref_id,
                            target_start=t_start + 1,  # Convert to 1-based index
                            target_end=t_end,
                            mismatches=errors,
                            query_sequence=probe_seq,
                            target_sequence=target_seq
                        ))
                    pbar.update(1)  # Update progress bar

                    # Reverse complement search
                    rev_ref_seq = str(Seq(ref_seq).reverse_complement())
                    adapter = BackAdapter(probe_seq, max_errors=max_errors, min_overlap=len(probe_seq), indels=False)
                    aligner = adapter.aligner

                    for t_start, t_end, errors, target_seq in find_all(rev_ref_seq, aligner):
                        results.append(Alignment(
                            query_name=probe_id,
                            target_name=f"{ref_id}(reverse)",
                            target_start=t_start + 1,
                            target_end=t_end,
                            mismatches=errors,
                            query_sequence=probe_seq,
                            target_sequence=target_seq
                        ))
                    pbar.update(1)  # Update progress bar again for reverse search

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Save results to file if requested
    if output_file:
        results_df.to_csv(output_file, index=False)
        sys.stderr.write(f"Results saved to {output_file}\n")

    return results_df


def find_all(ref, aligner):
    """Find all occurrences of a probe in a reference sequence."""
    offset = 0
    while True:
        result = aligner.locate(ref)
        if result is None:
            break
        ref_start, ref_end, query_start, query_end, score, errors = result
        t_start = query_start + offset
        t_end = query_end + offset
        target_seq = ref[query_start:query_end]
        yield (t_start, t_end, errors, target_seq)
        offset += query_start + 1
        ref = ref[query_start + 1:]


def find_targets(selected_features, fasta_file, reference_fasta, plp_length=30, min_coverage=1,
                 output_file='Candidate_probes', gc_min=50, gc_max=65, num_probes=10,
                 iupac_mismatches=None, max_errors=1, check_specificity=False):
    """
    Extract target sequences based on defined probe criteria and optionally check specificity.

    Args:
        selected_features (str): Path to the selected features file (TSV format).
        fasta_file (str): Path to the indexed FASTA file.
        output_file (str): Path to the output file.
        plp_length (int): Probe length (default: 30).
        min_coverage (int): Minimum coverage of the region.
        gc_min (int): Minimum GC content (default: 50).
        gc_max (int): Maximum GC content (default: 65).
        num_probes (int): Number of probes to select per gene (default: 10).
        iupac_mismatches (position:base): 
            List of positions (1-based) and IUPAC codes to introduce mismatches.        
            Recommended positions for mismatches are **15 and 16**.
            Example: 15:R,16:G
            **Note:** The total number of mismatches must be ≤2. Exceeding this limit may lead to unexpected behavior.

        max_errors (int): Maximum mismatches allowed during specificity checking.
        check_specificity (bool): Whether to check probe specificity against reference.

    Returns:
        DataFrame: DataFrame with extracted probe sequences (and specificity results if enabled).
    """

    targets = []

    seq_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    selected_features = pd.read_csv(selected_features, sep='\t')
    selected_features.index = selected_features['region']
    coverage_dict = dict(zip(selected_features['region'], selected_features['coverage']))

    if plp_length % 2 != 0:
        raise InputValueError("The size of the probe should be an even number", field="plp_length", code="odd_value_for_plp_length_provided")


    for keys in seq_dict:
        gene, region = keys.split("|")

        chr_name, start_end = region.split(":")
        initial_start = int(start_end.split("-")[0])
        seq = seq_dict[keys].seq
        seq_len = len(seq) - (plp_length - 1)

        coverage_value = coverage_dict.get(region, 0)
        if coverage_value < min_coverage:
            continue 

        for i in range(0, seq_len):
            tmp_seq = seq[i:i + plp_length]
            gc_content = (tmp_seq.count("G") + tmp_seq.count("C")) / len(tmp_seq) * 100

            if not (gc_min <= gc_content <= gc_max):
                continue
            if not any(nucleotide * 3 in tmp_seq for nucleotide in "ACGT"):
                continue

            start = initial_start + i
            end = start + plp_length - 1

            targets.append({
                "Probe_id": f"{gene}|{start}-{end}",
                "Gene": gene,
                "Region": f"{chr_name}:{start}-{end}",
                "Sequence": str(tmp_seq),
                "GC": gc_content,
                "Coverage": coverage_value,
                "Transcript_id": selected_features.loc[region, 'transcript_id']
            })

    targets_df = pd.DataFrame(targets)

    # Introduction of IUPAC mismatches
    if iupac_mismatches:
        targets_df = evaluate_ligation_junction(targets_df, iupac_mismatches=iupac_mismatches, plp_length=plp_length)


    # Check probe specificity against reference genome if requested
    if check_specificity:
        specificity_results = find_probes_in_targets(targets_df, reference_fasta, max_errors, output_file=f"{output_file}_specificity.csv")

        # Identify probes that have **only one unique match**
        unique_queries = specificity_results.groupby('query_name').size().reset_index(name='counts')
        unique_queries = unique_queries[unique_queries['counts'] == 1]['query_name']

        # Select only probes with a unique match
        targets_df = targets_df[targets_df['Probe_id'].isin(unique_queries)]

        # Merge mismatches into targets_df
        targets_df = targets_df.merge(
            specificity_results[['query_name', 'mismatches']],
            left_on='Probe_id',
            right_on='query_name',
            how='left'
        ).drop(columns=['query_name'])  # Drop duplicate column

        print(f"✅ Specificity results saved to {output_file}_specificity.csv")

    return targets_df



def parse_specificity_results(specificity_results):
    """
    Parse the specificity results from the CSV file.

    Args:
        specificity_results (str): Path to the specificity results CSV file.

    Returns:
        DataFrame: DataFrame containing parsed specificity results.
    """
    specificity_results.group_by('Probe_id').size().reset_index(name='counts')
    return pd.read_csv(specificity_results)

def evaluate_ligation_junction(targets, iupac_mismatches=None, plp_length=30, num_probes=10):
    """
    Evaluates the ligation junction of a probe and introduces mismatches if needed.

    Args:
        probe_seq (str): The probe sequence.
        iupac_mismatches (position:base): 
            List of positions (1-based) and IUPAC codes to introduce mismatches.        
            Recommended positions for mismatches are **15 and 16**.
            Example: 15:R,16:G
            **Note:** The total number of mismatches must be ≤2. Exceeding this limit may lead to unexpected behavior.
            
        plp_length (int): Probe length (default: 30).

    Returns:
        tuple: (updated probe sequence, ligation junction category)
    """
    # Add Ligation junction column to the DataFrame
    if 'Ligation junction' not in targets.columns:
        targets['Ligation junction'] = 'non-preferred'


    # Extract the ligation junction (2 bases around the center of the probe)
    junction_position = int((plp_length / 2) - 1)
    new_rows = []
    for idx in targets.index:

        probe_seq = targets.loc[idx]['Sequence']
        ligation_junction = probe_seq[junction_position] + probe_seq[junction_position + 2]

        # Determine the ligation status
        ligation_status = ligation_junctions_dict.get(ligation_junction, "non-preferred")
        targets.loc[idx, 'Ligation junction'] = ligation_status

        # Apply IUPAC mismatches if provided
        if iupac_mismatches is not None:
            if isinstance(iupac_mismatches, str):  # Only parse if it's a string
                iupac_mismatches = parse_iupac_mismatches(iupac_mismatches)

            num_mismatches = len(iupac_mismatches)

            if num_mismatches <= 2: # Limit to 2 mismatches
                for r in range(1, num_mismatches + 1): 
                    for subset in itertools.combinations(range(num_mismatches), r): # Generate all possible combinations of mismatches
                        selected_mismatches = [iupac_mismatches[i] for i in subset]
                        replacement_options = [IUPAC_CODES[symbol] for pos, symbol in selected_mismatches]

                        # Generate all possible combinations of replacements
                        for replacement in itertools.product(*replacement_options):
                            new_probe_seq = list(probe_seq)
                            new_id_suffix = []
                            for (pos, iupac_symbol), new_base in zip(selected_mismatches, replacement):
                                new_probe_seq[pos] = new_base
                                new_id_suffix.append(f"{pos}_{iupac_symbol}_{new_base}")
                            new_probe_seq = "".join(new_probe_seq)
                            new_probe_id = f"{targets.loc[idx, 'Probe_id']}|{'_'.join(new_id_suffix)}"
                            # Revaluate the ligation junction
                            new_ligation_junction = new_probe_seq[junction_position] + new_probe_seq[junction_position + 2]
                            new_ligation_status = ligation_junctions_dict.get(new_ligation_junction, "non-preferred")
                            new_row = targets.loc[idx].copy()
                            new_row['Sequence'] = new_probe_seq
                            new_row['Ligation junction'] = new_ligation_status
                            new_row['Probe_id'] = new_probe_id
                            new_rows.append((new_probe_id, new_row))
            else:
                raise InputValueError("The number of mismatches should be less than or equal to 2", field="iupac_mismatches", code="mismatches_exceed_limit")
   
    # Append the new rows to the DataFrame
    new_rows_df = pd.DataFrame([row[1] for row in new_rows], index=[row[0] for row in new_rows])
    targets = pd.concat([targets, new_rows_df])
    return targets

def parse_iupac_mismatches(mismatch_str):
    """
    Parses a string of mismatches formatted as "pos:base,pos:base" into a list of tuples.
    
    Args:
        mismatch_str (str): Mismatch input string (e.g., "5:R,10:G")
    
    Returns:
        list: A list of (position, base) tuples, e.g., [(5, 'R'), (10, 'G')].
    """
    mismatches = []
    try:
        for pair in mismatch_str.split(","):
            pos, base = pair.split(":")
            pos = int(pos.strip())  # Convert position to integer
            base = base.strip().upper()  # Ensure base is uppercase
            mismatches.append((pos, base))

    except (ValueError, IndexError):
        raise InputValueError("Invalid format for --iupac_mismatches. Use 'pos:base,pos:base', e.g., '5:R,10:G'.", 
                              field="iupac_mismatches", code="invalid_mismatch_format")
    
    return mismatches


# Melting temperature

## Oligo designer toolsuite

In [ ]:

#targets_df = pd.read_csv('../targets.txt', sep='\t')
#targets_df.set_index('Probe_id', inplace=True)
specificity_results = pd.read_csv('../probes_check.csv', sep=',')
selected_features = '../extract_features_output.txt'
#print(selected_features.head())

targets_df = targets_df_check = find_targets(
    selected_features=selected_features,
    fasta_file='../extract_seqs_output.fa',
    reference_fasta="../data/tmp_mrna.fa",
    plp_length=30,
    min_coverage=1,
    gc_min=50, gc_max=60,
    max_errors=1, check_specificity=True
)




In [ ]:
#!pip install oligo-designer-toolsuite
from oligo_designer_toolsuite.oligo_efficiency_filter import WeightedTmGCOligoScoring
import os
import pprint

from pathlib import Path

from oligo_designer_toolsuite.database import (
    OligoAttributes,
    OligoDatabase,
)
# Define scoring parameters
Tm_min = 55.0  # Minimum acceptable Tm
Tm_opt = 60.0  # Optimal Tm
Tm_max = 65.0  # Maximum acceptable Tm
GC_content_opt = 50.0  # Optimal GC content percentage

# Initialize the scoring object
scorer = WeightedTmGCOligoScoring(Tm_min=Tm_min, Tm_opt=Tm_opt, Tm_max=Tm_max, 
                                  GC_content_min = 50, GC_content_max = 60,
                                  Tm_parameters={},
                                  GC_content_opt=GC_content_opt)

min_oligos_per_region = 3
write_regions_with_insufficient_oligos = True
n_jobs = 3
lru_db_max_in_memory=n_jobs * 2 + 2
database_name="db_oligos"
oligo_database = OligoDatabase(
    min_oligos_per_region=min_oligos_per_region,
    write_regions_with_insufficient_oligos=write_regions_with_insufficient_oligos,
    lru_db_max_in_memory=lru_db_max_in_memory,
    database_name=database_name,
    dir_output='../oligo_database',
    n_jobs=n_jobs,
)

# Assuming 'oligo_database' is your database of probes and 'region_id' specifies the target region
#oligo_database, scores = scorer.apply(oligo_database, region_id='target_region')
oligo_database.load_database_from_fasta('Probes.txttmp.fa',
                                        database_overwrite=True,
                                        sequence_type='target')
fasta_file = 'Probes.txttmp.fa'
fasta_records = SeqIO.parse(fasta_file, "fasta")
probes_df = pd.DataFrame([{
    "id": record.id,
    "sequence": str(record.seq),
    "gc_content": gc_fraction(record.seq),
    "scores": int(0),
} for record in fasta_records])
probes_df.set_index('id', inplace=True)

scores_list = []
for record in probes_df.index:
    oligo_database, scores = scorer.apply(oligo_database, region_id=record, sequence_type='target')
    print(scores[0])
    scores_list.append(scores[0])  # Assuming scores[0] contains the relevant score

probes_df['scores'][:8] = scores_list
probes_df

Output()

/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_35010/3895664849.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(scores[0])
/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_35010/3895664849.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  scores_list.append(scores[0])  # Assuming scores[0] contains the relevant score
/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_35010/3895664849.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To ac

1.859
2.124
2.781
2.575
2.276
1.951
2.46
2.224


/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_35010/3895664849.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(scores[0])
/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_35010/3895664849.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  scores_list.append(scores[0])  # Assuming scores[0] contains the relevant score


ZeroDivisionError: float division by zero

## Python function based on `WeightedTmGCOligoScoring`

In [1]:
from Bio.SeqUtils import gc_fraction

def weighted_tm_gc_scoring(sequence, Tm_oligo, Tm_min=55.0, Tm_opt=60.0, Tm_max=65.0, 
                           GC_min=40.0, GC_opt=50.0, GC_max=60.0, w_Tm=1.0, w_GC=1.0):
    """
    Compute the weighted score for a given oligo based on melting temperature and GC content.
    
    Parameters:
    - sequence (str): The nucleotide sequence of the oligo.
    - Tm_oligo (float): The computed melting temperature of the oligo.
    - Tm_min, Tm_opt, Tm_max (float): Min, optimal, and max melting temperature thresholds.
    - GC_min, GC_opt, GC_max (float): Min, optimal, and max GC content thresholds.
    - w_Tm, w_GC (float): Weights for the Tm and GC components of the score.

    Returns:
    - float: The weighted score (lower is better).
    """
    # Compute GC content
    GC_oligo = gc_fraction(sequence) * 100  # Convert fraction to percentage

    # Compute Tm deviation score
    if Tm_oligo >= Tm_opt:
        score_Tm = abs(Tm_oligo - Tm_opt) / (Tm_max - Tm_opt)
    else:
        score_Tm = abs(Tm_oligo - Tm_opt) / (Tm_opt - Tm_min)
    
    # Compute GC deviation score
    if GC_oligo >= GC_opt:
        score_GC = abs(GC_oligo - GC_opt) / (GC_max - GC_opt)
    else:
        score_GC = abs(GC_oligo - GC_opt) / (GC_opt - GC_min)
    
    # Compute final weighted score
    score = w_Tm * score_Tm + w_GC * score_GC
    return score


## Comparison of both methods

In [ ]:
scores_list = []
for record in probes_df.index:
    oligo_seq = probes_df.loc[record]['sequence']

    # Compute Tm using nearest-neighbor method
    Tm_oligo = mt.Tm_NN(oligo_seq)

    # Compute the weighted score
    score = weighted_tm_gc_scoring(oligo_seq, Tm_oligo)
    scores_list.append(score)  # Assuming scores[0] contains the relevant score

probes_df['scores_python'] = scores_list

# calculate correlation between the two scores
probes_df[['scores', 'scores_python']][:8].corr()

NameError: name 'Tm_oligo' is not defined

## Implementing for each arms

In [ ]:
import numpy as np
from Bio.SeqUtils import gc_fraction, MeltingTemp as mt
import matplotlib.pyplot as plt
import tqdm

def weighted_tm_gc_scoring(sequence, Tm_oligo, Tm_min=55.0, Tm_opt=60.0, Tm_max=65.0, 
                           GC_min=40.0, GC_opt=50.0, GC_max=60.0, w_Tm=1.0, w_GC=1.0):
    """Compute the weighted score for a given oligo based on melting temperature and GC content."""
    GC_oligo = gc_fraction(sequence) * 100  # Convert fraction to percentage

    # Compute Tm deviation score
    if Tm_oligo >= Tm_opt:
        score_Tm = abs(Tm_oligo - Tm_opt) / (Tm_max - Tm_opt)
    else:
        score_Tm = abs(Tm_oligo - Tm_opt) / (Tm_opt - Tm_min)
    
    # Compute GC deviation score
    if GC_oligo >= GC_opt:
        score_GC = abs(GC_oligo - GC_opt) / (GC_max - GC_opt)
    else:
        score_GC = abs(GC_oligo - GC_opt) / (GC_opt - GC_min)
    
    return w_Tm * score_Tm + w_GC * score_GC  # Lower score is better


def analyze_scores(scores, percentile=5):
    """Analyze the score distribution and suggest a cutoff."""
    suggested_cutoff = np.percentile(scores, percentile)  # Get the threshold for top X%
    return suggested_cutoff


def score_padlock_probe(sequence, Tm_min=55.0, Tm_opt=60.0, Tm_max=65.0, 
                        GC_min=40.0, GC_opt=50.0, GC_max=60.0, w_Tm=1.0, w_GC=1.0, percentile=5):
    """Score a padlock probe by splitting it into two arms and analyzing distribution."""
    mid = len(sequence) // 2
    left_arm, right_arm = sequence[:mid], sequence[mid:]

    # Compute Tm for each arm
    Tm_left = mt.Tm_NN(left_arm)
    Tm_right = mt.Tm_NN(right_arm)

    # Score each arm separately
    score_left = weighted_tm_gc_scoring(left_arm, Tm_left, Tm_min, Tm_opt, Tm_max, GC_min, GC_opt, GC_max, w_Tm, w_GC)
    score_right = weighted_tm_gc_scoring(right_arm, Tm_right, Tm_min, Tm_opt, Tm_max, GC_min, GC_opt, GC_max, w_Tm, w_GC)

    # Combine scores (you can take the average, max, or another approach)
    final_score = (score_left + score_right) / 2  # Averaging the scores

    return final_score


def visualize_score_distribution(scores, cutoff):
    """Plot the score distribution and cutoff."""
    plt.figure(figsize=(8,5))
    plt.hist(scores, bins=30, alpha=0.7, color='blue', edgecolor='black')
    plt.axvline(cutoff, color='red', linestyle='dashed', linewidth=2, label=f'Cutoff ({cutoff:.2f})')
    plt.xlabel("Score")
    plt.ylabel("Frequency")
    plt.title("Distribution of Probe Scores")
    plt.legend()
    plt.show()


In [33]:
probes_df = pd.read_csv('../targets.txt', sep='\t')
sequences = probes_df['Sequence']
scores = [score_padlock_probe(seq) for seq in sequences]
probes_df['scores'] = scores
# Analyze distribution and suggest a cutoff
suggested_cutoff = analyze_scores(scores, percentile=5)
probes_df[probes_df['scores'] <= suggested_cutoff].shape[0]

#  visualize_score_distribution(scores, suggested_cutoff)

# print(f"Suggested cutoff based on top 5%: {suggested_cutoff:.2f}")

#probes_df = probes_df[probes_df['scores'] <= suggested_cutoff]
# Select probes with scores below the cutoff
#selected_probes = [seq for seq, score in zip(sequences, scores) if score <= suggested_cutoff]
#print(f"Selected probes: {selected_probes}")
#print(suggested_cutoff)

#probes_df = probes_df[probes_df['scores'] <= suggested_cutoff]


195

In [ ]:
min_distance = 10 # Define the minimum distance

def filter_probes_by_distance(group, min_distance):
    # Extract start and end coordinates from the 'Region' column
    group['Start'] = group['Region'].apply(lambda x: int(x.split(':')[1].split('-')[0]))
    group['End'] = group['Region'].apply(lambda x: int(x.split(':')[1].split('-')[1]))
    
    # Sort by start coordinate
    group = group.sort_values(by='Start')
    
    # Filter probes based on the minimum distance
    filtered_probes = []
    last_end = -min_distance  # Initialize with a value that ensures the first probe is included
    
    for idx, row in group.iterrows():
        if row['Start'] - last_end >= min_distance:
            filtered_probes.append(row)
            last_end = row['End']
    
    return pd.DataFrame(filtered_probes)

def filter_by_junction(probes_df)
# Apply the filtering function to each group
filtered_probes_df = probes_df.groupby('Gene').apply(lambda x: filter_probes_by_distance(x, min_distance)).reset_index(drop=True)

    

/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_15111/2949173634.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_probes_df = probes_df.groupby('Gene').apply(lambda x: filter_probes_by_distance(x, min_distance)).reset_index(drop=True)


,Probe_id,Gene,Region,Sequence,GC,Coverage,Transcript_id,Ligation junction,scores,Start,End
0,Grik2|48977243-48977272,Grik2,10:48977243-48977272,ATGTCTCTGAAGTGCCAGCGTCGGCTCAAA,53.333333,2.000000,ENSMUST00000218823;ENSMUST00000105484,neutral,3.794080,48977243,48977272
1,Grik2|48977323-48977352,Grik2,10:48977323-48977352,GCACACATTTAACGACAGAAGGTTGCCAGG,50.000000,2.000000,ENSMUST00000218823;ENSMUST00000105484,neutral,4.560182,48977323,48977352
2,Grik2|48989540-48989569|5_R_A,Grik2,10:48989540-48989569,GAGAAATGGTGGCGAGGCAATGGCTGCCCA,63.333333,6.000000,ENSMUST00000218823;ENSMUST00000218598;ENSMUST0...,preferred,3.487615,48989540,48989569
3,Grik2|48989579-48989608|10_G_G,Grik2,10:48989579-48989608,AGCAAAGAGGGCAGTGCTCTAGGGGTGCAG,60.000000,6.000000,ENSMUST00000218823;ENSMUST00000218598;ENSMUST0...,neutral,3.812583,48989579,48989608
4,Grik2|48989631-48989660|5_R_A,Grik2,10:48989631-48989660,TCCTGACAGCCGGCTTGGTGCTCTCAGTTT,60.000000,6.000000,ENSMUST00000218823;ENSMUST00000218598;ENSMUST0...,neutral,3.700053,48989631,48989660
5,Grik2|49008802-49008831,Grik2,10:49008802-49008831,ATGTGGGCATTTATGAGCAGCAGGAGACAG,50.000000,6.000000,ENSMUST00000218823;ENSMUST00000218598;ENSMUST0...,non-preferred,4.591969,49008802,49008831
6,Grik2|49008847-49008876|5_R_G,Grik2,10:49008847-49008876,AGCAAGGAGGAAGGGATTCAACGTGTCCTC,50.000000,6.000000,ENSMUST00000218823;ENSMUST00000218598;ENSMUST0...,non-preferred,4.075390,49008847,49008876
7,Grik2|49008899-49008928,Grik2,10:49008899-49008928,TGGAGTCAACGACCATCGAGTTTGTTACCC,50.000000,6.000000,ENSMUST00000218823;ENSMUST00000218598;ENSMUST0...,neutral,3.830444,49008899,49008928
8,Grik2|49008947-49008976,Grik2,10:49008947-49008976,CGCAGATTGGTGGCCTTATAGACTCCAAAG,50.000000,6.000000,ENSMUST00000218823;ENSMUST00000218598;ENSMUST0...,neutral,5.521361,49008947,49008976
9,Grik2|49116675-49116704,Grik2,10:49116675-49116704,GTTCTGAGCTCATGCCCAAAGCACTCTCCA,53.333333,6.000000,ENSMUST00000218823;ENSMUST00000218598;ENSMUST0...,neutral,3.749270,49116675,49116704


In [36]:
filtered_probes_df.shape

(38, 11)